In [16]:
import pandas as pd
import numba
import numpy as np

In [17]:
arr = pd.DataFrame({'v': [1, 2, 3], 'w': [4, 5, 6]}).to_records()
dtype = np.dtype([('a', 'f8'), ('b', 'f4'), ('c', 'f8')])

In [18]:
import pandas as pd
import numba
from numba import literal_unroll
import numpy as np

@numba.njit
def kernel(arr):
    return np.nanmean(arr['v']), np.nanmean(arr['w']), np.nanmean(arr['v'] + arr['w'])

dtype = np.dtype([('a', 'f8'), ('b', 'f4'), ('c', 'f8')])
fields = tuple(dtype.fields)

lower_index = np.array([0, 1, 2])
upper_index = np.array([1, 2, 3])

@numba.njit
def _loop(kernel, arr, lower_index, upper_index, dtype):
    results = np.zeros(len(arr), dtype=dtype)
    for i in range(len(arr)):
        result = kernel(arr[lower_index[i]: upper_index[i]])
        idx = 0
        for key in literal_unroll(fields):
            results[i][key] = result[idx]
            idx += 1
    return results


def loop(kernel, arr, lower_index, upper_index, dtype):
    fields = dtype.fields
    return _loop(kernel, arr, lower_index, upper_index, dtype)


loop(kernel, arr, lower_index, upper_index, dtype)

array([(1., 4., 5.), (2., 5., 7.), (3., 6., 9.)],
      dtype=[('a', '<f8'), ('b', '<f4'), ('c', '<f8')])